# Example experiment

Let's launch a "Hello world" experiment in three different languages! Here we leverage the slurm arrayjob feature to run the same job in parallel.

In [ ]:
# Experiment configuration

project_name = "example" # Choose a name for your project

main = "example.py" # The main script to run

# This is a "GridConfig" structure, which represents multiple configurations to evaluate for our "example.py" file.
config = {
    "string": [ # example.py takes a --string argument, here the array of values tells the runner to run the script with each of these values
        "Hello, World!",
        "Ciao Mondo!",
        "Merhaba Dünya!",
    ]
}

# This is a list of configuration names, which will help us identify the logs later
config_names = [
    "example", # This string will be concatenated to each configuration name
    [
        "en",  # This will make the 1st configuration name "example_en"
        "it",  # This will make the 2nd configuration name "example_it"
        "tr",  # This will make the 3rd configuration name "example_tr"
    ]
]

In [ ]:
# Environment configuration
# Here we can define additional arguments for the template

# since we are using a slurm template we can define the slurm configuration, may vary depending on the cluster
slurm_config = {
    "time": "10:00:00",
    "mem": "64G",
    "cpus-per-task": "2",
    "gres": "gpu:a100:1"
}

# This will be run before the experiment, useful for setting up the environment
run_before = f"""
source $HOME/venvs/base/bin/activate
"""

# Check out the template file for more arguments

### Run the experiment

In [ ]:
from hpctools import runners as r
from hpctools.utils import GridConfig, flatten_names

# Here we need to use the appropriate runner, since the cluster login node is accessible via SSH we use the SSHRunner
runner = r.SSHRunner(
    project_name=project_name,
    remote_workdir=f"/home/myhome/workspace/{project_name}", # This is the remote directory where workspace for this project will be created
    username="example",
    key_filename="~/.ssh/example",
    hostname="hpc.example.it",
)
runner.connect()


In [ ]:
import os
job = runner.run(
    run_name=os.path.basename(globals()['__vsc_ipynb_file__']).rsplit(".", 1)[0], # Get the jupyter file name, works only for vscode jupyter server
    template="../templates/slurm/arrayjob.slurm.j2",
    template_args={ # You can pass additional arguments to the template here
        "slurm_args": slurm_config,
        "run_before": run_before,
        "script_config_names": flatten_names(config_names),
        "script_configs": GridConfig(config).flatten(to_args=True),
        "script": f"python3 {main}", # This is the command that will be run on the cluster
        "run_after": "",
    }
)

### Monitoring

In [ ]:
# Visualize the slurm queue
from hpctools.jupyter_utils import SlurmQueueWidget

# This will display the slurm queue widget inside the jupyter notebook
SlurmQueueWidget(runner, job.name).view()